In [2]:
import MDAnalysis as mda
import nglview as nv


# Create two copy of the same universe
traj_clone_1 = mda.Universe('MDAnalysis_Programmation_python_2/PDB_tubuline.pdb',
                 'MDAnalysis_Programmation_python_2/trajectoire_tubuline.dcd')

traj_clone_2 = traj_clone_1.copy()

c:\Users\martb\miniconda3\envs\mdanalysis\Lib\site-packages\MDAnalysis\topology\TPRParser.py:161: DeprecationWarning: 'xdrlib' is deprecated and slated for removal in Python 3.13
  import xdrlib


c:\Users\martb\miniconda3\envs\mdanalysis\Lib\site-packages\MDAnalysis\coordinates\DCD.py:165: DeprecationWarning: DCDReader currently makes independent timesteps by copying self.ts while other readers update self.ts inplace. This behavior will be changed in 3.0 to be the same as other readers. Read more at https://github.com/MDAnalysis/mdanalysis/issues/3889 to learn if this change in behavior might affect you.
  warnings.warn("DCDReader currently makes independent timesteps"


In [3]:
from MDAnalysis.analysis import align


# Fit a target structure on a reference structure
align.AlignTraj(traj_clone_2,  # trajectory to align
                traj_clone_1,  # reference
                select='name CA',  # selection of atoms to align
                filename='aligned.dcd',  # file to write the trajectory to
                match_atoms=True,  # whether to match atoms based on mass
               ).run()

c:\Users\martb\miniconda3\envs\mdanalysis\Lib\site-packages\MDAnalysis\coordinates\DCD.py:448: UserWarning: No dimensions set for current frame, zeroed unitcell will be written
  warnings.warn(wmsg)


In [4]:
# Create a universe of a trajectory with frames aligned on reference
aligned = mda.Universe('MDAnalysis_Programmation_python_2/PDB_tubuline.pdb', 'aligned.dcd')

# Rename the segments for a different coloring treatment
aligned.segments.segids = ['Moving']
aligned.atoms.chainIDs = "A"
traj_clone_1.segments.segids = ['Static']
traj_clone_1.atoms.chainIDs = "B"

# Merged the reference and aligned universes
merged = mda.Merge(aligned.atoms, traj_clone_1.atoms)

In [5]:
import numpy as np
from MDAnalysis.analysis.base import AnalysisFromFunction
from MDAnalysis.coordinates.memory import MemoryReader


# Get the position of atoms of a structure
def copy_coords(ag):
    return ag.positions.copy()

# Iterate copy_coords() over each frame of the trajectory
aligned_coords = AnalysisFromFunction(copy_coords,
                                      aligned.atoms).run().results

# Get the number of frames
n_frames = aligned_coords['timeseries'].shape[0]

In [6]:
# Copy the coordinates of the first frame
first_coords = traj_clone_1.atoms.positions.copy()
first_coords.shape

(6924, 3)

In [7]:
# Create a static trajectory for reference
first_traj_coords = np.stack([first_coords] * n_frames)
first_traj_coords.shape

(1000, 6924, 3)

In [8]:
# Stack the reference and aligned trajectories 
merged_coords = np.hstack([aligned_coords['timeseries'],
                           first_traj_coords])
merged.load_new(merged_coords, format=MemoryReader)

<Universe with 13848 atoms>

In [9]:
# Create an ngl view of the merged universe
merged_view = nv.show_mdanalysis(merged, default_representation=False)
merged_view.add_cartoon(selection=":A and 420-450" , color = 'green')
merged_view.add_cartoon(selection=":A and 0-420" , color = 'green', opacity=0.6)
merged_view.add_cartoon(selection=":B", color = 'red', opacity=0.4)
merged_view

NGLWidget(max_frame=999)

In [10]:
from nglview.contrib.movie import MovieMaker

movie = MovieMaker(
    merged_view,
    step=100,  # only render every 4th frame
    output='merged.gif',
    render_params={"factor": 3},  # set to 4 for higher quality
)
movie.make()

IntProgress(value=0, description='Rendering ...', max=9)

In [11]:
# load the tubuline PDB file and display viewer widget
pdb_view = nv.show_file("MDAnalysis_Programmation_python_2/PDB_tubuline.pdb")
pdb_view.center()

pdb_view

NGLWidget()

In [12]:
univ_view = nv.show_mdanalysis(u)
univ_view.center()

univ_view

NameError: name 'u' is not defined

In [15]:
# select the first frame of the trajectory
traj_clone_1.trajectory[0]

# select the last frame of the trajectory
traj_clone_2.trajectory[-1] 

# merge the two structures
ex_frames = mda.Merge(traj_clone_1.atoms, traj_clone_2.atoms)

ex_frames_view = nv.show_mdanalysis(ex_frames)
ex_frames_view

NGLWidget()

In [ ]:
# Select the residues you want to color
selection = u.select_atoms('resid 400:450')

# Create an empty NGLView representation
viewt = nv.show_mdanalysis(selection, default_representation=False)

# Add cartoon representations for the selected residues
viewt.add_cartoon(color="blue", selection="440-450")
viewt.add_cartoon(color="red", selection="420-440")
viewt.add_balls_and_sticks()

# Center the view on the structure
viewt.center()

viewt

In [ ]:
print("NGLView version:", nv.__version__)
for attr in dir(nv):
    if attr.startswith("show_"):
        print(f"  nv.{attr}(...)")

In [ ]:
for attr in dir(viewt):
    if attr.startswith("add_"):
        print(f"view.{attr}(...)")

In [ ]:
test = nv.show_mdanalysis(u)
# help(nv.color)
# print(dir(nv.color.COLOR_SCHEMES))
?test.shape.add_sphere